In [30]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [31]:
url = 'https://ca.finance.yahoo.com/crypto'
response = requests.get(url)
page_content = response.text
doc = BeautifulSoup(page_content, 'html.parser')

# Sentiment analysis on news header

In [41]:
# Model for sentiment analysis
topic_model = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
label = pipeline('sentiment-analysis', model = topic_model)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [42]:
header = []
news_elements = doc.find_all('a', {'class': "js-content-viewer"})
for i in range (len(news_elements)):
    header.append(news_elements[i].text)
news_header_df = pd.DataFrame({'News_Header': header})

In [43]:
classification = []
for i in range (len(news_header_df.News_Header)):
    classification.append(label(news_header_df.News_Header[i]))

In [44]:
labels = []
for i in range (len(classification)):
    labels.append(classification[i][0]['label'])

In [47]:
news_header_df['sentiment_analysis'] = labels
news_header_df['positive count'] = (news_header_df['sentiment_analysis']).count('positvie')
news_header_df['neutral count'] = (news_header_df['sentiment_analysis']).count('neutral')
news_header_df['negative count'] = (news_header_df['sentiment_analysis']).count('negative')

TypeError: Series.count() takes 1 positional argument but 2 were given

In [38]:
news_header_df

,News_Header,sentiment_analysis
0,MARKETS LIVE BLOGSTOCKS EXTEND LOSSES AS YIELD...,positive
1,Bitcoin Could Fall Another 25%. Will Elon Musk...,negative
2,Bitcoin Price Falls,negative
3,Bitcoin Plunge Spurs Liquidations as SpaceX’s ...,neutral
4,Bitcoin drops to new two-month low as world ma...,negative
5,"No, SpaceX Isn't the Cause of This Multi-Billi...",neutral
6,Wintermute Proposal To Borrow YFI Draws Fire F...,neutral
7,Bitcoin price heads higher as US CPI data cont...,positive
8,UK watchdog raids 34 sites in crypto ATM crack...,neutral
9,Bitcoin rally falters as crypto pulls back yet...,negative


# Web scrape market data

In [ ]:
simpTblRow Bgc($hoverBgColor):h BdB Bdbc($seperatorColor